# Porting genome scale metabolic models for metabolomics

**Worm-GEM as default worm model, for better compatibility**
https://github.com/SysBioChalmers/Worm-GEM

**Use cobra to parse SBML models whereas applicable**

Not all models comply with the formats in cobra. Models from USCD and Thiele labs should comply.

**Base our code on metDataModel**

Each model needs a list of Reactions, list of Pathways, and a list of Compounds.
It's important to include with Compounds with all linked identifiers to other DBs (HMDB, PubChem, etc), and with formulae (usually charged form in these models) when available.
We can alwasy update the data later. E.g. the neural formulae can be inferred from charged formula or retrieved from public metabolite database (e.g., HMDB) if linked.
Save in Python pickle and in JSON.

**No compartmentalization**
- After decompartmentalization,
  - transport reactions can be removed - they are identified by reactants and products being the same.
  - redundant reactions can be merge - same reactions in diff compartments become one.

Shuzhao Li, 2021-10-21|
Minghao Gong, 2022-04-25

In [1]:
# !pip install cobra --user --ignore-installed ruamel.yaml
# !pip install --upgrade metDataModel # https://github.com/shuzhao-li/metDataModel/ 
# !pip install --upgrade numpy pandas

In [2]:
import cobra # https://cobrapy.readthedocs.io/en/latest/io.html#SBML
from metDataModel.core import Compound, Reaction, Pathway, MetabolicModel
import requests
import sys

sys.path.append("/Users/gongm/Documents/projects/mass2chem/")
sys.path.append("/Users/gongm/Documents/projects/JMS/JMS/JMS")
from mass2chem.formula import *
from jms.formula import *
from jms.utils.gems import *

In [3]:
# download the most updated Worm-GEM.xml
model_name = 'Worm-GEM'
xml_url = f'https://github.com/SysBioChalmers/{model_name}/blob/main/model/{model_name}.xml'
local_path = f'../testdata/{model_name}/'

try:
    os.mkdir(local_path)
except:
    None

xml_file_name = f'{model_name}.xml'
git_download_from_file(xml_url,local_path,xml_file_name)

In [4]:
# Read the model via cobra
xmlFile = os.path.join(local_path,xml_file_name)
model = cobra.io.read_sbml_model(xmlFile)

'' is not a valid SBML 'SId'.
https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [5]:
model

Name,
Memory address,0x07f8f9588ef10
Number of metabolites,8150
Number of reactions,12187
Number of groups,150
Objective expression,1.0*MAR00021 - 1.0*MAR00021_reverse_97974
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [6]:
# metabolite entries, readily convert to list of metabolites
model.metabolites[990] 

Metabolite identifier,MAM00616p
Name,25(R)THCA-CoA
Memory address,0x07f8f775c2ee0
Formula,C48H76N7O20P3S
Compartment,p
In 3 reaction(s),"MAR01625, MAR02285, MAR01632"


In [7]:
# reaction entries, Readily convert to list of reactions
model.reactions[33]

Reaction identifier,MAR04122
Name,
Memory address,0x07f8f48023ca0
Stoichiometry,MAM02040c + 2.0 MAM02552c + MAM03108c --> 3.0 MAM02039c + 2.0 MAM02553c + MAM03109c H2O + 2.0 NAD+ + UDP-glucose --> 3.0 H+ + 2.0 NADH + UDP-glucuronate
GPR,sqv-4
Lower bound,0.0
Upper bound,1000.0


# There are no group/pathway information in this 

## Port metabolite

In [8]:
def port_metabolite(M):
    # convert cobra Metabolite to metDataModel Compound
    Cpd = Compound()
    Cpd.src_id = remove_compartment_by_substr(M.id,1)
    Cpd.id = remove_compartment_by_substr(M.id,1)              # temporarily the same with the source id
    Cpd.name = M.name
    Cpd.charge = M.charge
    Cpd.neutral_formula = adjust_charge_in_formula(M.formula,M.charge)
    Cpd.neutral_mono_mass = neutral_formula2mass(Cpd.neutral_formula)
    Cpd.charged_formula = M.formula
    Cpd.db_ids = [[model_name,Cpd.src_id]] # using src_id to also reference Worm-GEM ID in db_ids field
    for k,v in M.annotation.items():
        if k != 'sbo':
            if isinstance(v,list):
                Cpd.db_ids.append([[k,x] for x in v])
            else: 
                if ":" in v:
                    Cpd.db_ids.append([k,v.split(":")[1]])
                else:
                    Cpd.db_ids.append([k,v])
    
    inchi_list = [x[1].split('=')[1] for x in Cpd.db_ids if x[0] == 'inchi']
    if len(inchi_list) ==1:
        Cpd.inchi = inchi_list[0]
    elif len(inchi_list) >1:
        Cpd.inchi = inchi_list
        
    return Cpd

In [9]:
myCpds = []
for i in range(len(model.metabolites)):
    myCpds.append(port_metabolite(model.metabolites[i]))

In [10]:
len(myCpds)

8150

In [11]:
# remove duplicated compounds
myCpds = remove_duplicate_cpd(myCpds)

In [12]:
myCpds[100].__dict__

{'internal_id': '',
 'id': 'MAM00101',
 'name': '(5Z,8Z,11Z)-eicosatrienoyl-CoA',
 'db_ids': [['Worm-GEM', 'MAM00101']],
 'neutral_formula': 'C41H68N7O17P3S',
 'neutral_mono_mass': 1055.360526,
 'charge': -4,
 'charged_formula': 'C41H64N7O17P3S',
 'SMILES': '',
 'inchi': '',
 'src_id': 'MAM00101'}

In [13]:
len(myCpds)

4032

In [14]:
fetch_MetabAtlas_GEM_identifiers(compound_list = myCpds,
                                 modelName = model_name,
                                 local_path = local_path,
                                 metab_file_name = 'metabolites.tsv',
                                 overwrite = True)

In [15]:
myCpds[100].__dict__

{'internal_id': '',
 'id': 'MAM00101',
 'name': '(5Z,8Z,11Z)-eicosatrienoyl-CoA',
 'db_ids': [('HMR2', 'm00101c'),
  ('LipidMaps', 'LMFA07050062'),
  ('MetaNetX', 'MNXM47372'),
  ('MetaNetX', 'MNXM488534'),
  ('Recon3D', 'M00101')],
 'neutral_formula': 'C41H68N7O17P3S',
 'neutral_mono_mass': 1055.360526,
 'charge': -4,
 'charged_formula': 'C41H64N7O17P3S',
 'SMILES': '',
 'inchi': '',
 'src_id': 'MAM00101'}

## Port reactions

In [16]:
# port reactions, to include genes and enzymes
def port_reaction(R):
    new = Reaction()
    new.id = R.id
    new.reactants = [remove_compartment_by_substr(m.id,1) for m in R.reactants] # decompartmentalization
    new.products = [remove_compartment_by_substr(m.id,1) for m in R.products]   # decompartmentalization
    new.genes = [g.id for g in R.genes]
    ecs = R.annotation.get('ec-code', [])
    if isinstance(ecs, list):
        new.enzymes = ecs
    else:
        new.enzymes = [ecs]       # this version of Worm-GEM may have it as string
    return new

test99 = port_reaction(model.reactions[199])
[test99.id,
 test99.reactants,
 test99.products,
 test99.genes,
 test99.enzymes
]

['MAR04002',
 ['MAM01285'],
 ['MAM01334', 'MAM01371'],
 ['F13E6.2', 'let-754', 'ZK673.2', 'F38B2.4', 'taf-9'],
 ['2.7.4.3']]

In [17]:
## Reactions to port
myRxns = []
for R in model.reactions:
    myRxns.append( port_reaction(R) )
    
print(len(myRxns))

12187


In [18]:
# remove duplicated reactions after decompartmentalization
myRxns = remove_duplicate_rxn(myRxns)

In [19]:
len(myRxns)

8047

In [20]:
myRxns[0].__dict__

{'azimuth_id': '',
 'id': 'MAR03905',
 'source': [],
 'version': '',
 'status': '',
 'reactants': ['MAM01796', 'MAM02552'],
 'products': ['MAM01249', 'MAM02039', 'MAM02553'],
 'enzymes': ['1.1.1.1', '1.1.1.71'],
 'genes': ['adh-5', 'hphd-1'],
 'pathways': [],
 'ontologies': [],
 'species': '',
 'compartments': [],
 'cell_types': [],
 'tissues': []}

## Port pathway

In [21]:
# pathways, using group as pathway. Other models may use subsystem etc.

def port_pathway(P):
    new = Pathway()
    new.id = P.id
    new.source = ['Worm-GEM v1.10.0',]
    new.name = P.name
    new.list_of_reactions = [x.id for x in P.members]
    return new

p = port_pathway(model.groups[12])

[p.id, p.name, p.list_of_reactions[:5]]

['group13',
 'Ascorbate and aldarate metabolism',
 ['MAR06393', 'MAR06394', 'MAR06396', 'MAR06405', 'MAR08345']]

In [22]:
## Pathways to port
myPathways = []
for P in model.groups:
    myPathways.append(port_pathway(P))

len(myPathways)

150

In [23]:
# retain the valid reactions in list of pathway
myPathways = retain_valid_Rxns_in_Pathways(myPathways,myRxns)

In [24]:
# test if the length of unique reactions matched with the length of decompartmentalized reaction list 
test_list_Rxns = []
for pathway in myPathways:
    for y in pathway.list_of_reactions:
        test_list_Rxns.append(y)

len(set(test_list_Rxns))

8047

## Collected data; now output

In [32]:
from datetime import datetime
today =  str(datetime.today()).split(" ")[0]

In [33]:
today

'2022-04-25'

In [34]:
note = f"""{model_name} compartmentalized, with genes and ECs."""

## metabolicModel to export
MM = MetabolicModel()
MM.id = f'az_{model_name}_{today}' #
MM.meta_data = {
            'species': model_name.split('-')[0],
            'version': '',
            'sources': [f'https://github.com/SysBioChalmers/{model_name}, retrieved {today}'], #
            'status': '',
            'last_update': today,  #
            'note': note,
        }
MM.list_of_reactions = [R.serialize() for R in  myRxns]
MM.list_of_compounds = [C.serialize() for C in myCpds]
MM.list_of_pathways = [P.serialize() for P in myPathways]

In [35]:
# check output
[
MM.list_of_reactions[:2],
MM.list_of_compounds[100:102],
MM.list_of_pathways[:2]
]

[[{'id': 'MAR03905',
   'reactants': ['MAM01796', 'MAM02552'],
   'products': ['MAM01249', 'MAM02039', 'MAM02553'],
   'genes': ['adh-5', 'hphd-1'],
   'enzymes': ['1.1.1.1', '1.1.1.71']},
  {'id': 'MAR03907',
   'reactants': ['MAM01796', 'MAM02554'],
   'products': ['MAM01249', 'MAM02039', 'MAM02555'],
   'genes': ['Y39G8B.1'],
   'enzymes': ['1.1.1.2']}],
 [{'id': 'MAM00101',
   'name': '(5Z,8Z,11Z)-eicosatrienoyl-CoA',
   'identifiers': [('HMR2', 'm00101c'),
    ('LipidMaps', 'LMFA07050062'),
    ('MetaNetX', 'MNXM47372'),
    ('MetaNetX', 'MNXM488534'),
    ('Recon3D', 'M00101')],
   'neutral_formula': 'C41H68N7O17P3S',
   'charge': -4,
   'charged_formula': 'C41H64N7O17P3S',
   'neutral_mono_mass': 1055.360526,
   'SMILES': '',
   'inchi': ''},
  {'id': 'MAM00102',
   'name': '(5Z,8Z,11Z,14Z,17Z)-eicosapentaenoylcarnitine',
   'identifiers': [('BiGG', 'tmndnccrn'),
    ('HMR2', 'm00102c'),
    ('MetaNetX', 'MNXM9158'),
    ('Recon3D', 'tmndnccrn')],
   'neutral_formula': 'C27H43NO

In [36]:
import pickle
import os

# Write pickle file
export_pickle(os.path.join(local_path,f'{MM.id}.pickle'), MM)

In [37]:
# Write json file
export_json(os.path.join(local_path,f'{MM.id}.json'), MM)

In [38]:
# Write dataframe 
import pandas as pd
export_table(os.path.join(local_path,f'{MM.id}_list_of_compounds.csv'),MM, 'list_of_compounds')
export_table(os.path.join(local_path,f'{MM.id}_list_of_reactions.csv'),MM, 'list_of_reactions')
export_table(os.path.join(local_path,f'{MM.id}_list_of_pathways.csv'),MM, 'list_of_pathways')

## Summary

This ports reactions, pathways and compounds. Gene and enzyme information is now included. 

The exported pickle can be re-imported and uploaded to Database easily.

This notebook, the pickle file and the JSON file go to GitHub repo (https://github.com/shuzhao-li/Azimuth).